In [3]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00462/drugsCom_raw.zip
!unzip drugsCom_raw.zip
!rm *.zip

Archive:  drugsCom_raw.zip
  inflating: drugsComTest_raw.tsv    
  inflating: drugsComTrain_raw.tsv   


In [0]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_recall_fscore_support

In [0]:
df = pd.read_csv('drugsComTrain_raw.tsv', sep='\t')
test_df = pd.read_csv('drugsComTest_raw.tsv', sep='\t')

In [5]:
df.head()

,Unnamed: 0,drugName,condition,review,rating,date,usefulCount
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9.0,"May 20, 2012",27
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8.0,"April 27, 2010",192
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5.0,"December 14, 2009",17
3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8.0,"November 3, 2015",10
4,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9.0,"November 27, 2016",37


In [0]:
# data = df['review']
# labels = df['rating'].astype(int)

In [0]:
# data.head()
# labels.head()

In [0]:
df['rating'] = df['rating'].astype(int)
test_df['rating'] = test_df['rating'].astype(int)

## convert to three classes

In [0]:
def helper(a):
    if a<=4:
        return 0
    elif a>=7:
        return 2
    else: return 1

df['sent'] = df['rating']
df['sent'] = df.sent.apply(helper)

test_df['sent'] = test_df['rating']
test_df['sent'] = test_df.sent.apply(helper)

In [0]:
# df.head()
# df.loc[250]

## bag of words

In [0]:
x_train = df['review']
x_test = test_df['review']

vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.75, stop_words='english', lowercase=True)
x_train = vectorizer.fit_transform(x_train)
x_test = vectorizer.transform(x_test)

### for 10-class

In [0]:
# for 10-class
y_train = df['rating']
y_test = test_df['rating']

### for 3-class

In [0]:
# for 3-class
y_train = df['sent']
y_test = test_df['sent']

# MODELS:

## Logistic regression

In [12]:
%%time

lr_clf = LogisticRegression(random_state =1, solver='lbfgs', multi_class='multinomial', class_weight='balanced')
lr_clf.fit(x_train, y_train)
pred = lr_clf.predict(x_test)
score = metrics.accuracy_score(y_test, pred)
print('Accuracy is ', 100*score)

Accuracy is  71.31830524867017
CPU times: user 27.4 s, sys: 19.8 s, total: 47.3 s
Wall time: 23.9 s


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


In [0]:
# precision_recall_fscore_support(y_test, pred)

In [13]:
print(metrics.classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.66      0.70      0.68     13497
           1       0.24      0.53      0.33      4829
           2       0.91      0.74      0.82     35440

    accuracy                           0.71     53766
   macro avg       0.60      0.66      0.61     53766
weighted avg       0.79      0.71      0.74     53766



## Random forest classifier

In [14]:
%%time

clf = RandomForestClassifier() # class_weight='balanced'
clf.fit(x_train, y_train)
pred = clf.predict(x_test)
score = metrics.accuracy_score(y_test, pred)
print('Accuracy is ', 100*score)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Accuracy is  87.98311200386861
CPU times: user 1min 27s, sys: 73.3 ms, total: 1min 28s
Wall time: 1min 28s


In [0]:
# precision_recall_fscore_support(y_test, pred)

In [0]:
print(metrics.classification_report(y_test, pred))

             precision    recall  f1-score   support

          1       0.66      0.79      0.72      7299
          2       0.85      0.64      0.73      2334
          3       0.87      0.63      0.73      2205
          4       0.89      0.62      0.73      1659
          5       0.85      0.61      0.71      2710
          6       0.90      0.58      0.71      2119
          7       0.86      0.58      0.69      3091
          8       0.76      0.62      0.68      6156
          9       0.73      0.65      0.69      9177
         10       0.68      0.88      0.77     17016

avg / total       0.75      0.73      0.72     53766

